In [1]:
import import_ipynb

from statistics import mean
from DataSet import *

importing Jupyter notebook from DataSet.ipynb


In [2]:
net.to("cuda:0")
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

In [ ]:
for epoch in range(5):
    loss_log = []
    for x, y in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        o = net(x)
        
        loss = loss_f(o, y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        loss_log.append(loss.item())
        
    test_score = eval_net(net, test_loader, device="cuda:0")
    print(epoch, mean(loss_log), test_score, flush=True)

 70%|██████▉   | 12298/17579 [02:42<01:19, 66.79it/s]

In [ ]:
# Move Trained-Model to 'CPU'.
net.to("cpu")